## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Bluff,-46.6000,168.3333,overcast clouds,55.00,80,89,21.99,NZ
1,1,Kieta,-6.2167,155.6333,overcast clouds,80.51,80,99,3.24,PG
2,2,Carnarvon,-24.8667,113.6333,few clouds,75.20,83,12,16.11,AU
3,3,North Haven,41.3909,-72.8595,clear sky,60.80,25,1,14.97,US
4,4,Ilulissat,69.2167,-51.1000,light snow,21.20,63,75,19.57,GL


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,1,Kieta,-6.2167,155.6333,overcast clouds,80.51,80,99,3.24,PG
2,2,Carnarvon,-24.8667,113.6333,few clouds,75.20,83,12,16.11,AU
7,7,Tiarei,-17.5333,-149.3333,scattered clouds,84.20,74,40,4.61,PF
11,11,Ransang,8.9603,117.5828,overcast clouds,77.04,81,85,3.85,PH
12,12,Puerto Ayora,-0.7393,-90.3518,few clouds,86.00,66,20,16.11,EC
14,14,Rikitea,-23.1203,-134.9692,broken clouds,77.52,68,79,17.49,PF
18,18,Hithadhoo,-0.6000,73.0833,overcast clouds,83.30,73,92,18.03,MV
22,22,Faanui,-16.4833,-151.7500,few clouds,82.08,75,20,9.91,PF
23,23,Kapaa,22.0752,-159.3190,broken clouds,75.20,60,75,14.97,US
25,25,Ciudad Bolivar,8.1222,-63.5497,overcast clouds,81.19,70,98,15.48,VE


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                263
City                   263
Lat                    263
Lng                    263
Current Description    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Country                263
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("","NaN", inplace=True)
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kieta,PG,80.51,overcast clouds,-6.2167,155.6333,
2,Carnarvon,AU,75.20,few clouds,-24.8667,113.6333,
7,Tiarei,PF,84.20,scattered clouds,-17.5333,-149.3333,
11,Ransang,PH,77.04,overcast clouds,8.9603,117.5828,
12,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,
14,Rikitea,PF,77.52,broken clouds,-23.1203,-134.9692,
18,Hithadhoo,MV,83.30,overcast clouds,-0.6000,73.0833,
22,Faanui,PF,82.08,few clouds,-16.4833,-151.7500,
23,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,
25,Ciudad Bolivar,VE,81.19,overcast clouds,8.1222,-63.5497,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("","NaN", inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kieta,PG,80.51,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
2,Carnarvon,AU,75.20,few clouds,-24.8667,113.6333,Hospitality Carnarvon
7,Tiarei,PF,84.20,scattered clouds,-17.5333,-149.3333,Le Rocher de Tahiti
11,Ransang,PH,77.04,overcast clouds,8.9603,117.5828,Black and Crystal Sand Campsite
12,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
708,Tomatlan,MX,87.60,clear sky,19.9333,-105.2500,Norma franco
709,Phuket,TH,86.00,few clouds,8.0000,98.2500,The Surin Phuket
712,El Tigre,VE,86.11,overcast clouds,8.8875,-64.2454,Hotel Santa Cruz
714,Florianopolis,BR,77.00,scattered clouds,-27.6146,-48.5012,Hotel Boutique Quinta das Videiras


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))